 # Populate database

Example notebook to illustrace how to populate the database using SQLModel with a postgres data.

## Packages and options

In [1]:
from sqlmodel import SQLModel, Field, create_engine, Session, delete, select
import pandas as pd
import os

## Establish the database connection

In [2]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
db = os.getenv("POSTGRES_DB")
uri = f"postgresql://{user}:{password}@{host}/{db}"
engine = create_engine(uri)

## Simple model

Have a look at the data:

In [3]:
fn = "../data/extracted/transformation/01-01-electricity_consumption.csv"
df = pd.read_csv(fn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8540 entries, 0 to 8539
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sector    8540 non-null   object 
 1   year      8540 non-null   int64  
 2   value     8540 non-null   float64
 3   scenario  8540 non-null   object 
 4   variant   8540 non-null   object 
 5   unit      8540 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 400.4+ KB


For the first shot, do not think about normalization but just create a model with the corresponding table in the database:

In [4]:
class ElectricityConsumption(SQLModel, table=True):
    id: int = Field(primary_key=True)
    scenario: str
    variant: str
    sector: str
    year: int
    value: float
    unit: str

Create the table (if it does not exist already)

In [5]:
SQLModel.metadata.create_all(engine)

Add the data to the database. To avoid duplicates by running the cell twice, we delete the data before injecting them:

In [6]:
with Session(engine) as session:
    # delete the data
    session.exec(delete(ElectricityConsumption))
    records = df.to_dict(orient="records")
    session.add_all([
        ElectricityConsumption(**r) for r in records
    ])
    session.commit()

Get the data again. We get back a list of ElectricityConsumption instances (in fact pytantic model instances) that we convert to a dataframe:

In [7]:
with Session(engine) as session:
    res = session.exec(select(ElectricityConsumption)).all()
df_out = pd.DataFrame([r.model_dump() for r in res])
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8540 entries, 0 to 8539
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   scenario  8540 non-null   object 
 1   variant   8540 non-null   object 
 2   year      8540 non-null   int64  
 3   unit      8540 non-null   object 
 4   sector    8540 non-null   object 
 5   id        8540 non-null   int64  
 6   value     8540 non-null   float64
dtypes: float64(1), int64(2), object(4)
memory usage: 467.2+ KB


## More structured

The model above is simple but might not be able to discover that different tables have different thinks in common. For this it might be useful to build relations between the data. One appraohc could be to introduce a relation data model by extracting the different diemnsions of the model. In the case of the consumption data, we might extract the sector as well as the scenario-variant combinations into own tables that will be referenced by several variables.

In [8]:
class Scenario(SQLModel, table=True):
    id: int = Field(primary_key=True)
    scenario: str
    variant: str

class Sector(SQLModel, table=True):
    id: int = Field(primary_key=True)
    sector: str

class ElecConsumption(SQLModel, table=True):
    id: int = Field(primary_key=True)
    scenario_id: int = Field(default=None, foreign_key="scenario.id")
    sector_id: int = Field(default=None, foreign_key="sector.id")
    year: int
    value: float
    unit: str

Create the tables (if not exist):

In [9]:
SQLModel.metadata.create_all(engine)

As we now introduced references, we need to create the reference data first and also need to use the identifiers in our consumptions data:

In [10]:
df_sector = df[["sector"]].drop_duplicates()
df_scenario = df[["scenario", "variant"]].drop_duplicates()

In [11]:
with Session(engine) as session:
    # delete the data to avoid duplicates if cell is run twice
    # note that we need to first delete the table that uses the references...
    session.exec(delete(ElecConsumption))
    session.exec(delete(Scenario))
    session.exec(delete(Sector))
    # add the records
    session.add_all([Sector(**r) for r in df_sector.to_dict(orient="records")])
    session.add_all([Scenario(**r) for r in df_scenario.to_dict(orient="records")])
    session.commit()
    # get the data again to create mappings to insert the id into the consumption data
    df_sector_db = (
        pd.DataFrame([r.model_dump() for r in session.exec(select(Sector)).all()])
        .rename(columns={"id": "sector_id"})
    )
    df_scenario_db = (
        pd.DataFrame([r.model_dump() for r in session.exec(select(Scenario)).all()])
        .rename(columns={"id": "scenario_id"})
    )    

Insert the foreign keys into the consumption data and insert the resulting table into the database:

In [12]:
df_insert = (
    df.merge(df_sector_db, on="sector", how="left").drop(columns=["sector"])
    .merge(df_scenario_db, on=["scenario", "variant"], how="left").drop(columns=["scenario", "variant"])
)
with Session(engine) as session:
    # add the records
    session.add_all([ElecConsumption(**r) for r in df_insert.to_dict(orient="records")])
    session.commit()  
    # get the orignal data back joining the tables
    statement = select(
        ElecConsumption,
        Scenario.scenario,
        Scenario.variant,
        Sector.sector
    ).join(
        Scenario, ElecConsumption.scenario_id == Scenario.id
    ).join(
        Sector, ElecConsumption.sector_id == Sector.id
    )
    df_out = pd.read_sql(statement, session.bind)
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8540 entries, 0 to 8539
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8540 non-null   int64  
 1   scenario_id  8540 non-null   int64  
 2   sector_id    8540 non-null   int64  
 3   year         8540 non-null   int64  
 4   value        8540 non-null   float64
 5   unit         8540 non-null   object 
 6   scenario     8540 non-null   object 
 7   variant      8540 non-null   object 
 8   sector       8540 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 600.6+ KB


In [13]:
df_out

,id,scenario_id,sector_id,year,value,unit,scenario,variant,sector
0,51297,51,85,2056,62.847994,TWh,ZERO-Basis,KKW50,EEV Strom
1,51298,51,85,2057,62.723762,TWh,ZERO-Basis,KKW50,EEV Strom
2,51299,51,85,2058,62.601761,TWh,ZERO-Basis,KKW50,EEV Strom
3,51300,51,85,2059,62.475753,TWh,ZERO-Basis,KKW50,EEV Strom
4,51301,51,85,2060,62.325072,TWh,ZERO-Basis,KKW50,EEV Strom
...,...,...,...,...,...,...,...,...,...
8535,59776,60,98,2056,78.138307,TWh,WWB,KKW60,Bruttoverbrauch
8536,59777,60,98,2057,78.445425,TWh,WWB,KKW60,Bruttoverbrauch
8537,59778,60,98,2058,78.907732,TWh,WWB,KKW60,Bruttoverbrauch
8538,59779,60,98,2059,79.237788,TWh,WWB,KKW60,Bruttoverbrauch
